# Exercise 1

## Imports

In [1]:
import pickle
import os.path
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, ArrayType, FloatType, DoubleType
from itertools import combinations
from typing import Iterable, Any, List, Set

import pandas as pd
import numpy as np
from typing import Union

## Spark initialization

In [2]:
spark = SparkSession.builder \
    .appName('exercise1') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

23/04/20 21:52:34 WARN Utils: Your hostname, martinho-MS-7B86 resolves to a loopback address: 127.0.1.1; using 192.168.1.67 instead (on interface enp34s0)
23/04/20 21:52:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/20 21:52:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Prepare the data

In [3]:
tracks_df = (spark.read
    .option("multiline", "true")
    .option("quote", '"')
    .option("escape", '"')
    .csv('data/tracks.csv')
)

# rename columns with row values from first row to second row
column_categories = list(zip(*tracks_df.take(2)))
columns = tracks_df.columns
tracks_df = tracks_df.select(F.col(columns[0]).alias('track_id'),
    *(F.col(column).alias("-".join(map(str, categories)))
    for column, categories in zip(columns[1:], column_categories[1:]))
)

tracks_df = (tracks_df
    .filter(F.col("track_id").isNotNull()) 
    .filter(F.col("track_id") != "track_id")
)

tracks_df.show()

23/04/20 21:52:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+--------------+-------------------+-------------------+--------------+---------------+--------+--------------------+-------------+--------------------+----------+--------------------+------------+----------------+------------------------+----------------------+------------------------+--------------------+---------------+-------------------+----------------+---------+---------------+------------------+----------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------------------+---------+----------+--------------+--------------+--------------+-------------------+-------------------+--------------+---------------+---------------+------------+-----------------+--------------------+--------------+-------------------+--------------------+-------------+--------------+------------+---------------+----------+--------------------+
|track_id|album-comments| album-date_created|album-date_released|album-engineer|album-

In [4]:
# TODO: feature selection? not needed necessarily but...
features_df = (spark.read
    .csv('data/features.csv')
)

# rename columns with row values from first row to second row
column_categories = list(zip(*features_df.take(3)))
columns = features_df.columns
# TODO: DoubleType instead of FloatType?
features_df = features_df.select(F.col(columns[0]).alias('track_id'),
    *(F.col(column).cast(DoubleType()).alias("-".join(map(str, categories)))
    for column, categories in zip(columns[1:], column_categories[1:]))
)

features_df = (features_df
    .filter(F.col("track_id") != "feature")
    .filter(F.col("track_id") != "statistics")
    .filter(F.col("track_id") != "number")
    .filter(F.col("track_id") != "track_id")
)

## Agglomerative clustering (in-memory)

In [5]:
dataset_subset_name = "small"

small_tracks_df = tracks_df.filter(F.col("set-subset") == dataset_subset_name)
small_features_df = (features_df
    .join(small_tracks_df, "track_id", "left")
    .filter(F.col("set-subset").isNotNull())
    .select(features_df.columns)
)

small_features_pd = small_features_df.toPandas()

In [6]:
# calculate the metrics (radius, diameter, density_r, density_d) for each cluster
def calculate_metrics(pd_df, centroids):
    
    cluster = pd_df["cluster"].values[0]
    metrics = pd.DataFrame({'radius': [0], 'diameter': [0],'density_r': [0],'density_d': [0]}, columns=['radius', 'diameter','density_r','density_d'])
    centroid = centroids[cluster].reshape(1,-1)

    matrix = pd_df.drop(columns=["cluster", "track_id"]).to_numpy()
    
    matrix_radius = np.sqrt(np.sum((matrix - centroid)**2, axis=1))
    metrics.loc[0,'radius'] = np.max(matrix_radius)
    # calculate density with radius
    metrics.loc[0,'density_r'] = len(pd_df) / metrics.loc[0,'radius']**2

    for i in range(matrix.shape[0]):
        matrix_diameter = np.sqrt(np.sum((matrix[i:,:] - matrix[i,:])**2, axis=1))

        max_diameter = np.max(matrix_diameter)
        if max_diameter > metrics.loc[0,'diameter']:
            metrics.loc[0,'diameter'] = max_diameter
        
    # calculate density with diameter
    metrics.loc[0,'density_d'] = len(pd_df) / metrics.loc[0,'diameter']**2

    return metrics

In [7]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
import numpy.typing as npt

metrics_pd_array = []

small_features_pd["cluster"] = np.zeros((len(small_features_pd), 1), dtype=np.int32)

def cluster_agglomeratively(data: pd.DataFrame, n_clusters: int) -> npt.NDArray[np.float32]:
    if "cluster" not in data.columns:
        raise ValueError("The 'cluster' column should be present in the dataframe!")
    
    data_features_only = data.drop(columns=["cluster", "track_id"])

    clusterer = AgglomerativeClustering(n_clusters=n_clusters)
    clusterer.fit(data_features_only)
    
    centroid_calculator = NearestCentroid()
    centroid_calculator.fit(data_features_only, clusterer.labels_)

    data["cluster"] = clusterer.labels_
    return centroid_calculator.centroids_

if os.path.exists("./results/metrics_pd_array_pickle.pkl"):
    with open("./results/metrics_pd_array_pickle.pkl",'rb') as f:
        metrics_pd_array = pickle.load(f)

else:
    # i = 8 until 16
    for n_clusters in range(8, 17):
        print(f"n_clusters: {n_clusters}")
        centroids = cluster_agglomeratively(small_features_pd, n_clusters)
        metrics_pd_array.append(small_features_pd.groupby("cluster").apply(calculate_metrics, centroids))

    # TODO: weird column at 0s without name??
    with open("./results/metrics_pd_array_pickle.pkl",'wb') as f:
        pickle.dump(metrics_pd_array, f)

/tmp/ipykernel_66721/1666919071.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  small_features_pd["cluster"] = np.zeros((len(small_features_pd), 1), dtype=np.int32)


In [8]:
# TODO: justify N-clusters choice, maybe create a colored matrix?
for i in metrics_pd_array:
    density_r_average = i["density_r"].mean()
    density_d_average = i["density_d"].mean()
    density_r_variance = i["density_r"].var()
    density_d_variance = i["density_d"].var()

    print(f"Average density_r: {density_r_average}, Average density_d: {density_d_average}, Variance density_r: {density_r_variance}, Variance density_d: {density_d_variance}")
    print("\n")

Average density_r: 7.962110575648213e-05, Average density_d: 3.599342497079214e-05, Variance density_r: 7.554281742039276e-09, Variance density_d: 1.632545349816937e-09


Average density_r: 8.46827574451412e-05, Average density_d: 3.664885346147574e-05, Variance density_r: 7.747059677670204e-09, Variance density_d: 1.6211171070541367e-09


Average density_r: 8.164132830338627e-05, Average density_d: 3.4647549186503005e-05, Variance density_r: 7.052526954162877e-09, Variance density_d: 1.4927145547250588e-09


Average density_r: 7.530009483773654e-05, Average density_d: 3.330844689045004e-05, Variance density_r: 5.65147072510877e-09, Variance density_d: 1.2235780221208591e-09


Average density_r: 7.0833141687739e-05, Average density_d: 3.124377322263076e-05, Variance density_r: 5.389143411471879e-09, Variance density_d: 1.1658465305734218e-09


Average density_r: 6.755260440629955e-05, Average density_d: 2.980317784207617e-05, Variance density_r: 5.035928633263812e-09, Variance density_

## BFR Algorithm

In [9]:
# TODO: results can be: density of clusters, number of nodes in each cluster, etc., but not strictly necessary

In [10]:
# TODO: why were 9 clusters chosen?
n_clusters = 9
dimensions = len(features_df.columns) - 1   # don't consider 'track_id'

max_memory_used_megabytes = 4000
# Assumes all columns are floats/integers, and so therefore 4 bytes
rows_per_iteration = max_memory_used_megabytes // (4 * len(features_df.columns))
total_rows = features_df.count()

seed_random = 0

### Initialize clusters

In [11]:
k_centroids = cluster_agglomeratively(small_features_pd, n_clusters)

### Loop

In [12]:
import dataclasses

@dataclasses.dataclass(init=False)
class SummarizedCluster:
    n:      int                     
    sum_:   npt.NDArray[np.float64]
    sumsq_: npt.NDArray[np.float64]
    id_:    Union[int, None]
    tracks: Set[int]

    def __init__(self, dimensions: int, id_: int=None):
        self.n = 0
        self.sum_ = np.zeros((dimensions,), dtype=np.float64)
        self.sumsq_ = np.zeros((dimensions,), dtype=np.float64)
        self.id_ = id_
        self.tracks = set()
    
    def summarize(self, point: npt.NDArray[np.float64], track_id: int):
        self.n += 1
        self.sum_ += point
        self.sumsq_ += point**2
        self.tracks.add(track_id)
    
    def summarize_points(self, points: npt.NDArray[np.float64], track_ids: Set[int]):
        self.n += points.shape[0]
        self.sum_ += np.sum(points, axis=0)
        self.sumsq_ += np.sum(points**2, axis=0)
        self.tracks |= track_ids
        self.test = points

    def centroid(self) -> npt.NDArray[np.float64]:
        return self.sum_ / self.n

    def variance(self) -> npt.NDArray[np.float64]:
        return (self.sumsq_ / self.n) - (self.sum_ / self.n)**2

    def standard_deviation(self) -> npt.NDArray[np.float64]:
        return np.sqrt(self.variance())

    def __add__(self, other) -> 'SummarizedCluster':
        if self.id_ is not None and self.other is not None and self.id_ != self.other:
            raise ValueError(f"Clusters {self} and {other} have different explicit ids ({self.id_} != {other.id_}).")
        res = SummarizedCluster(self.dimensions, self.id_ if self.id_ is not None else other.id_)
        res.n = self.n + other.n
        res.sum_ = self.sum_ + other.sum_
        res.sumsq_ = self.sumsq_ + other.sumsq_
        res.tracks = self.tracks | other.tracks
        return res

In [13]:
discard_sets: List[SummarizedCluster] = [SummarizedCluster(dimensions, id_) for id_ in range(n_clusters)]
compression_sets: List[SummarizedCluster] = []
retained_set: List[npt.NDArray[np.float32]] = []

Summarize clusters with the clustering of the small dataset subset.

In [14]:
def summarize_cluster_df(cluster_df: pd.DataFrame) -> None:
    cluster_id = cluster_df["cluster"].values[0]
    cluster_features_mtx = cluster_df.drop(columns=["cluster", "track_id"]).to_numpy()
    
    track_ids = set(cluster_df["track_id"].values)

    discard_set = discard_sets[cluster_id]
    discard_set.summarize_points(cluster_features_mtx, track_ids)

small_features_pd.groupby("cluster").apply(summarize_cluster_df)

""


Setup functions and thresholds necessary for BFR.

In [51]:
# Threshold in terms of standard deviations away from centroid, in each dimension
cluster_distance_threshold_standard_deviations = 1
cluster_distance_threshold = ((cluster_distance_threshold_standard_deviations**2) * dimensions) ** 0.5

# TODO: choose threshold and justify decision
compression_set_merge_variance_threshold = float('-inf')

dbscan_eps = 0.5    # TODO: parameter?

assert len(k_centroids) == n_clusters, "The number of clusters does not coincide with the number of random centroids!"

# def mahalanobis_distance_np(x: npt.NDArray[np.float32], s: SummarizedCluster) -> float:
#     return np.sqrt(np.sum(((x - s.centroid()) / s.standard_deviation())**2))

def mahalanobis_distance_pd(x: pd.DataFrame, s: SummarizedCluster) -> pd.Series:
    return (((x - s.centroid()) / s.standard_deviation())**2).sum(axis=1) ** 0.5

# @F.udf(returnType=FloatType())
# def closest_mahalanobis_distance_cluster(*features: float):
#     x = np.array(features)
#     closest_cluster_distance, closest_cluster = min((mahalanobis_distance_np(x, d), d.id_) for d in discard_sets)
#     return closest_cluster if closest_cluster_distance < cluster_distance_threshold else None

In [16]:
from sklearn.cluster import DBSCAN

features_columns = set(features_df.columns) - {"track_id"}
split_weights = [1.0] * (total_rows // rows_per_iteration)

features_without_small_df = (features_df
    .join(small_tracks_df, "track_id", "left")
    .filter(F.col("set-subset").isNull())
    .select(features_df.columns)
)

In [ ]:
(small_features_pd.drop(columns=["track_id", "cluster"]) + np.arange(518).reshape((518,)) * 1)

In [18]:
loaded_points_pd = small_features_pd.copy()

In [ ]:
d = discard_sets[0]

In [ ]:
len(d.test)

In [ ]:
np.where(d.variance() < 0)

In [ ]:
526.9983**2 / 527**2

In [ ]:
526.9966 / 527

In [ ]:
print(d.sum_[104], d.sumsq_[104])

In [30]:
loaded_points_pd.iloc[:,1:519].shape

(8000, 518)

In [ ]:
loaded_points_pd: pd.DataFrame

In [127]:
prefix = "cluster_distance_"
cluster_distance_columns = [f"{prefix}{i}" for i in range(n_clusters)]
loaded_points_pd[cluster_distance_columns] = np.ones((loaded_points_pd.shape[0], n_clusters))

for i, discard_set in enumerate(discard_sets):
    loaded_points_pd[f"cluster_distance_{i}"] = mahalanobis_distance_pd(loaded_points_pd.iloc[:, 1:519], discard_set)
loaded_points_pd["min_cluster_distance"] = loaded_points_pd[cluster_distance_columns].min(axis=1)
loaded_points_pd["cluster_id"] = loaded_points_pd[cluster_distance_columns].idxmin(axis=1).str.slice(start=len(prefix)).astype(np.int32)

loaded_points_pd.drop(columns=cluster_distance_columns, inplace=True)

# Don't consider the points that surpass the threshold for the discard sets
loaded_points_pd.loc[loaded_points_pd["min_cluster_distance"] >= cluster_distance_threshold - 4, "cluster_id"] = -1
loaded_points_pd.drop(columns=["min_cluster_distance"], inplace=True)

In [139]:
pd.concat([loaded_points_pd, pd.DataFrame(np.zeros((1, 522)), columns=loaded_points_pd.columns)], axis=0, ignore_index=True)

,track_id,chroma_cens-kurtosis-01,chroma_cens-kurtosis-02,chroma_cens-kurtosis-03,chroma_cens-kurtosis-04,chroma_cens-kurtosis-05,chroma_cens-kurtosis-06,chroma_cens-kurtosis-07,chroma_cens-kurtosis-08,chroma_cens-kurtosis-09,...,zcr-kurtosis-01,zcr-max-01,zcr-mean-01,zcr-median-01,zcr-min-01,zcr-skew-01,zcr-std-01,cluster,cluster_tmp,cluster_id
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448,20.204036,21.174570,-1.0
1,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861,17.233454,17.233454,8.0
2,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800,21.125072,22.857894,-1.0
3,140,0.533579,-0.623885,-1.086205,-1.081079,-0.765151,-0.072282,-0.882913,-0.582376,-0.884749,...,11.052547,0.379395,0.052379,0.036621,0.001953,3.143968,0.057712,15.630419,18.973435,3.0
4,141,0.172898,-0.284804,-1.169662,-1.062855,-0.706868,-0.708281,-0.204884,0.023624,-0.642770,...,32.994659,0.415527,0.040267,0.034668,0.002930,4.204097,0.028665,15.125494,24.417967,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7996,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7997,154413,-0.214509,-1.130469,0.718534,-0.368448,-0.147830,-0.099409,-1.325709,-0.105248,-1.363881,...,25.368595,0.323242,0.024532,0.018066,0.000977,3.736646,0.023821,22.169065,27.847208,-1.0
7998,154414,-0.487371,-0.923754,-0.283099,-0.435221,-1.137329,-0.798039,-0.258168,1.004049,-0.499121,...,21.276468,0.511230,0.046116,0.033691,0.003418,3.997052,0.045733,17.251695,24.705431,3.0
7999,155066,0.044216,-0.300441,-0.217022,-0.356106,-1.085789,-1.185135,-0.655948,-1.517006,-0.490595,...,116.044044,0.554199,0.016058,0.009766,0.000000,9.688635,0.030787,21.273305,33.545784,-1.0


In [ ]:
prefix = "cluster_distance_"
cluster_distance_columns = [f"{prefix}{i}" for i in range(n_clusters)]

def print_progress(progress: float, message: str):
    print(f"[{progress:3%}] {message:100}", end="\r")

print_progress(0, "Initialized BFR")
for split_idx, loaded_points_df in enumerate(features_without_small_df.randomSplit(split_weights, seed=seed_random)):
    progress = split_idx / len(split_weights)
    
    print_progress(progress, "Clustering with the Mahalanobis distance...")

    loaded_points_pd[cluster_distance_columns] = np.ones((loaded_points_pd.shape[0], n_clusters))

    for i, discard_set in enumerate(discard_sets):
        loaded_points_pd[f"cluster_distance_{i}"] = mahalanobis_distance_pd(loaded_points_pd.iloc[:, 1:519], discard_set)
    loaded_points_pd["min_cluster_distance"] = loaded_points_pd[cluster_distance_columns].min(axis=1)
    loaded_points_pd["cluster_id"] = loaded_points_pd[cluster_distance_columns].idxmin(axis=1).str.slice(start=len(prefix)).astype(np.int32)

    loaded_points_pd.drop(columns=cluster_distance_columns, inplace=True)

    # Don't consider the points that surpass the threshold for the discard sets
    loaded_points_pd.loc[loaded_points_pd["min_cluster_distance"] >= cluster_distance_threshold - 4, "cluster_id"] = -1
    loaded_points_pd.drop(columns=["min_cluster_distance"], inplace=True)

    # cluster_mapping = loaded_points_df \
    #     .withColumn("cluster", closest_mahalanobis_distance_cluster(*features_columns)) \
    #     .groupby("cluster") \
    #     .agg({
    #         "track_ids": F.collect_list("track_id"),
    #         "features_list": F.collect_list(F.array(*features_columns))
    #     }).collect()
    
    print_progress(progress, "Calculated and collected Mahalanobis distances")

    for cluster_id, cluster_df in loaded_points_pd.groupby("cluster"):
        track_ids = cluster_df["track_id"]
        features_list = cluster_df[:, 1:519]

        print_progress(progress, f"Evaluated cluster {cluster_id}...")

        # Step 3 - check which points go to the discard sets
        if cluster_id != -1:
            discard_set = discard_sets[cluster_id]
            for track_id, features in zip(track_ids, features_list):
                discard_set.summarize(np.array(features), track_id)
        
        # Step 4 - check which points go to the compression sets or the retained set
        else:
            matrix_to_cluster = np.array(features_list + retained_set)

            # Use same distance as above
            clusterer = DBSCAN(eps=dbscan_eps, metric='euclidean')
            clusterer.fit(matrix_to_cluster)

            centroid_calculator = NearestCentroid()
            centroid_calculator.fit(matrix_to_cluster, clusterer.labels_)

            retained_set.clear()

            # Create compression sets
            compression_sets_temp = [SummarizedCluster(dimensions, None) for _ in centroid_calculator.centroids_]

            for point_idx in range(matrix_to_cluster.shape[0]):
                cluster_id = clusterer.labels_[point_idx]
                point = matrix_to_cluster[point_idx, :]
                
                if cluster_id == -1:
                    retained_set.append(point)
                else:
                    compression_sets_temp[cluster_id].summarize(point)
            
            compression_sets.extend(compression_sets_temp)
        
    print_progress(progress, f"Finished evaluating clusters (compression sets: {len(compression_sets)}, retained set: {len(retained_set)})")
    
    # Step 5 - merge compression sets
    compressing = True
    while compressing:
        compressing = False
        merged_compression_sets = []
        compression_set_idxs_to_remove = set()

        for (idx_1, compression_set_1), (idx_2, compression_set_2) in combinations(enumerate(compression_sets), 2):
            if idx_1 in compression_set_idxs_to_remove or idx_2 in compression_set_idxs_to_remove:
                continue

            merged_compression_set = compression_set_1 + compression_set_2
            if merged_compression_set.variance() < compression_set_merge_variance_threshold:
                merged_compression_sets.append(merged_compression_set)
                compression_set_idxs_to_remove.add(idx_1)
                compression_set_idxs_to_remove.add(idx_2)
                compressing = True
        
        compression_sets = [cs for i, cs in enumerate(compression_sets) if i not in compression_set_idxs_to_remove]
        compression_sets.extend(merged_compression_sets)

        print_progress(progress, f"Completed one compression (compression sets: {len(compression_sets)})")
    
    print_progress((split_idx + 1) / len(split_weights), f"Finished one of the splits (compression sets: {len(compression_sets)}, retained set: {len(retained_set)})")

# Step 6 - merge CS and RS into DS (but we won't yet)
print()

## BFR cluster visualization